<img src="./images/DLI_Header.png" style="width: 400px;">

# Assessment

Congratulations on going through today's course! Hopefully, you've learned some valuable skills along the way and had fun doing it. Now it's time to put those skills to the test. In this assessment, you will train a new model that is able to recognize fresh and rotten fruit. You will need to get the model to a validation accuracy of `92%` in order to pass the assessment, though we challenge you to do even better if you can. You will have the use the skills that you learned in the previous exercises. Specifically, we suggest using some combination of transfer learning, data augmentation, and fine tuning. Once you have trained the model to be at least 92% accurate on the validation dataset, save your model, and then assess its accuracy. Let's get started! 

## The Dataset

In this exercise, you will train a model to recognize fresh and rotten fruits. The dataset comes from [Kaggle](https://www.kaggle.com/sriramr/fruits-fresh-and-rotten-for-classification), a great place to go if you're interested in starting a project after this class. The dataset structure is in the `data/fruits` folder. There are 6 categories of fruits: fresh apples, fresh oranges, fresh bananas, rotten apples, rotten oranges, and rotten bananas. This will mean that your model will require an output layer of 6 neurons to do the categorization successfully. You'll also need to compile the model with `categorical_crossentropy`, as we have more than two categories.

<img src="./images/fruits.png" style="width: 600px;">

## Load ImageNet Base Model

We encourage you to start with a model pretrained on ImageNet. Load the model with the correct weights, set an input shape, and choose to remove the last layers of the model. Remember that images have three dimensions: a height, and width, and a number of channels. Because these pictures are in color, there will be three channels for red, green, and blue. We've filled in the input shape for you. This cannot be changed or the assessment will fail. If you need a reference for setting up the pretrained model, please take a look at [notebook 05b](05b_presidential_doggy_door.ipynb) where we implemented transfer learning.

In [12]:
from tensorflow import keras

base_model = keras.applications.VGG16(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False)

## Freeze Base Model

Next, we suggest freezing the base model, as done in [notebook 05b](05b_presidential_doggy_door.ipynb). This is done so that all the learning from the ImageNet dataset does not get destroyed in the initial training.

In [13]:
# Freeze base model
base_model.trainable = False

## Add Layers to Model

Now it's time to add layers to the pretrained model. [Notebook 05b](05b_presidential_doggy_door.ipynb) can be used as a guide. Pay close attention to the last dense layer and make sure it has the correct number of neurons to classify the different types of fruit.

In [14]:
# Create inputs with correct shape
inputs = keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=False)

# Add pooling layer or flatten layer
x = keras.layers.GlobalAveragePooling2D()(x)

# Add final dense layer
outputs = keras.layers.Dense(6, activation = 'softmax')(x)

# Combine inputs and outputs to create model
model = keras.Model(inputs, outputs)

In [15]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 3078      
Total params: 14,717,766
Trainable params: 3,078
Non-trainable params: 14,714,688
_________________________________________________________________


## Compile Model

Now it's time to compile the model with loss and metrics options. Remember that we're training on a number of different categories, rather than a binary classification problem.

In [16]:
model.compile(loss = keras.losses.CategoricalCrossentropy(from_logits=True) , metrics = [keras.metrics.CategoricalAccuracy()])

## Augment the Data

If you'd like, try to augment the data to improve the dataset. Feel free to look at [notebook 04a](04a_asl_augmentation.ipynb) and [notebook 05b](05b_presidential_doggy_door.ipynb) for augmentation examples. There is also documentation for the [Keras ImageDataGenerator class](https://keras.io/api/preprocessing/image/#imagedatagenerator-class). This step is optional, but it may be helpful to get to 92% accuracy.

In [17]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


datagen = ImageDataGenerator(
        samplewise_center=True,  # set each sample mean to 0
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False) 

## Load Dataset

Now it's time to load the train and validation datasets. Pick the right folders, as well as the right `target_size` of the images (it needs to match the height and width input of the model you've created). If you'd like a reference, you can check out [notebook 05b](05b_presidential_doggy_door.ipynb).

In [18]:
# load and iterate training dataset
train_it = datagen.flow_from_directory('data/fruits/train/', 
                                       target_size=(224, 224), 
                                       color_mode='rgb', 
                                       class_mode="categorical")
# load and iterate validation dataset
valid_it = datagen.flow_from_directory('data/fruits/valid/', 
                                      target_size=(224, 224), 
                                      color_mode='rgb', 
                                      class_mode="categorical")

Found 1182 images belonging to 6 classes.
Found 329 images belonging to 6 classes.


## Train the Model

Time to train the model! Pass the `train` and `valid` iterators into the `fit` function, as well as setting your desired number of epochs.

In [19]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=30)

Epoch 1/30
37/36 [==============================] - 19s 526ms/step - loss: 1.5577 - categorical_accuracy: 0.4788 - val_loss: 1.3894 - val_categorical_accuracy: 0.6596
Epoch 2/30
37/36 [==============================] - 19s 526ms/step - loss: 1.3634 - categorical_accuracy: 0.6827 - val_loss: 1.3122 - val_categorical_accuracy: 0.7356
Epoch 3/30
37/36 [==============================] - 19s 526ms/step - loss: 1.2808 - categorical_accuracy: 0.7640 - val_loss: 1.2404 - val_categorical_accuracy: 0.8024
Epoch 4/30
37/36 [==============================] - 19s 526ms/step - loss: 1.2490 - categorical_accuracy: 0.8029 - val_loss: 1.2227 - val_categorical_accuracy: 0.8237
Epoch 5/30
37/36 [==============================] - 19s 524ms/step - loss: 1.2262 - categorical_accuracy: 0.8249 - val_loss: 1.1996 - val_categorical_accuracy: 0.8480
Epoch 6/30
37/36 [==============================] - 20s 530ms/step - loss: 1.2149 - categorical_accuracy: 0.8274 - val_loss: 1.2081 - val_categorical_accuracy: 0.838

In [22]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=1)

37/36 [==============================] - 19s 524ms/step - loss: 1.0558 - categorical_accuracy: 0.9882 - val_loss: 1.0708 - val_categorical_accuracy: 0.9757


In [10]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=30)

Epoch 1/30
37/36 [==============================] - 21s 560ms/step - loss: 1.5569 - categorical_accuracy: 0.7369 - val_loss: 0.7459 - val_categorical_accuracy: 0.8328
Epoch 2/30
37/36 [==============================] - 21s 573ms/step - loss: 0.6176 - categorical_accuracy: 0.8071 - val_loss: 0.8731 - val_categorical_accuracy: 0.7204
Epoch 3/30
37/36 [==============================] - 21s 566ms/step - loss: 0.5733 - categorical_accuracy: 0.8113 - val_loss: 1.0928 - val_categorical_accuracy: 0.6869
Epoch 4/30
37/36 [==============================] - 21s 567ms/step - loss: 1.8469 - categorical_accuracy: 0.7614 - val_loss: 0.6705 - val_categorical_accuracy: 0.8207
Epoch 5/30
37/36 [==============================] - 21s 576ms/step - loss: 0.7040 - categorical_accuracy: 0.8063 - val_loss: 0.6073 - val_categorical_accuracy: 0.8085
Epoch 6/30
37/36 [==============================] - 21s 556ms/step - loss: 0.6290 - categorical_accuracy: 0.8003 - val_loss: 0.6699 - val_categorical_accuracy: 0.756

In [11]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=30)

Epoch 1/30
37/36 [==============================] - 21s 558ms/step - loss: 1.5320 - categorical_accuracy: 0.8012 - val_loss: 0.7496 - val_categorical_accuracy: 0.8207
Epoch 2/30
37/36 [==============================] - 21s 565ms/step - loss: 0.4935 - categorical_accuracy: 0.8469 - val_loss: 1.6327 - val_categorical_accuracy: 0.7508
Epoch 3/30
37/36 [==============================] - 21s 566ms/step - loss: 0.6721 - categorical_accuracy: 0.8071 - val_loss: 0.5774 - val_categorical_accuracy: 0.8328
Epoch 4/30
37/36 [==============================] - 21s 570ms/step - loss: 1.2083 - categorical_accuracy: 0.7115 - val_loss: 0.5897 - val_categorical_accuracy: 0.7903
Epoch 5/30
37/36 [==============================] - 21s 571ms/step - loss: 0.7926 - categorical_accuracy: 0.7665 - val_loss: 1.0763 - val_categorical_accuracy: 0.7751
Epoch 6/30
37/36 [==============================] - 21s 565ms/step - loss: 0.5174 - categorical_accuracy: 0.8376 - val_loss: 0.9082 - val_categorical_accuracy: 0.829

## Unfreeze Model for Fine Tuning

If you have reached 92% validation accuracy already, this next step is optional. If not, we suggest fine tuning the model with a very low learning rate.

In [ ]:
# Unfreeze the base model
base_model.trainable = True

# Compile the model with a low learning rate
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = .00001),
              loss = keras.losses.CategoricalCrossentropy(from_logits=True) , metrics = [keras.metrics.CategoricalAccuracy()])

In [ ]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=100)

## Evaluate the Model

Hopefully, you now have a model that has a validation accuracy of 92% or higher. If not, you may want to go back and either run more epochs of training, or adjust your data augmentation. 

Once you are satisfied with the validation accuracy, evaluate the model by executing the following cell. The evaluate function will return a tuple, where the first value is your loss, and the second value is your accuracy. To pass, the model will need have an accuracy value of `92% or higher`. 

In [23]:
model.evaluate(valid_it, steps=valid_it.samples/valid_it.batch_size)

11/10 [================================] - 4s 339ms/step - loss: 1.0660 - categorical_accuracy: 0.9848


[1.0660316944122314, 0.9848024249076843]

## Run the Assessment

To assess your model run the following two cells.

**NOTE:** `run_assessment` assumes your model is named `model` and your validation data iterator is called `valid_it`. If for any reason you have modified these variable names, please update the names of the arguments passed to `run_assessment`.

In [24]:
from run_assessment import run_assessment

In [25]:
run_assessment(model, valid_it)

Evaluating model 5 times to obtain average accuracy...

11/10 [================================] - 4s 352ms/step - loss: 1.0801 - categorical_accuracy: 0.9635

Accuracy required to pass the assessment is 0.92 or greater.
Your average accuracy is 0.9708.

Congratulations! You passed the assessment!
See instructions below to generate a certificate.


## Generate a Certificate

If you passed the assessment, please return to the course page (shown below) and click the "ASSESS TASK" button, which will generate your certificate for the course.

<img src="./images/assess_task.png" style="width: 800px;">